In [35]:
import os
import pandas as pd

path = '/home/local/work/code/github/project-diploid-assembly/annotation/grch38/known_regions'
out_path = '/home/local/work/code/github/project-diploid-assembly/annotation/grch38'
names = ['Modeled_regions_for_GRCh38.tsv', 'GRCh38_p13_chromXY_PAR.tsv', 'ucsc_cytoband.bed']

header = [
    'name',
    'chrom',
    'start',
    'end',
    'length'
]

ucsc_header = [
    'chrom',
    'start',
    'end',
    'name',
    'gieStain'
]

sort_order = [
    'chrom',
    'start',
    'end',
    'name',
    'score',
]


chrom_file = 'hg38_hgsvc2_noalt.chroms.tsv'
chroms = pd.read_csv(os.path.join(path, chrom_file), sep='\t', names=['chrom', 'length'])

all_regions = []

for n, hd in zip(names, [header, header, ucsc_header]):
    print(n)
    file_path = os.path.join(path, n)
    df = pd.read_csv(file_path, sep='\t', names=hd, header=None, comment='#')
    if not 'ucsc' in n:
        # for BED output, convert to 0-based
        df['end'] += 1
        df['chrom'] = 'chr' + df['chrom']
    if 'ucsc' in n:
        df.loc[~df['name'].isna(), 'name'] = df['chrom'] + df['name']
        df.loc[df['name'].isna(), 'name'] = 'ctg'
        df = df.loc[df['chrom'].isin(chroms['chrom']), :].copy()
    else:
        df['name'] = df['name'].str.replace('#', '')
    df['strand'] = '.'
    df['score'] = 1000
    if 'ucsc' not in n:
        df.loc[df['name'].str.startswith('HET'), 'score'] = 750
        df.loc[df['name'].str.startswith('PAR'), 'score'] = 500

    if 'gieStain' in df.columns:
        df['gieStain_value'] = df['gieStain'].str.extract('([0-9]+)', expand=False)
        df.loc[df['gieStain_value'].isna(), 'gieStain_value'] = '0'
        df['gieStain_value'] = df['gieStain_value'].astype('int16')
        df['gieStain_value'] *= 10
        df['score'] = df['gieStain_value']

    df['score'] = df['score'].astype('int16')
    if 'ucsc' in n:
        out_cyto = os.path.join(out_path, 'GRCh38_cytobands.bed')
        df = df[sort_order]
        with open(out_cyto, 'w') as dump:
            _ = dump.write('#')
            df.to_csv(dump, sep='\t', header=True, index=False)
    else:
        all_regions.append(df)

all_regions = pd.concat(all_regions, axis=0)
all_regions.sort_values(['chrom', 'start', 'end'], inplace=True)
sort_order = [
    'chrom',
    'start',
    'end',
    'name',
    'score',
]

all_regions = all_regions[sort_order]
out_path = '/home/local/work/code/github/project-diploid-assembly/annotation/grch38'

# dump as BED file for intersect operations
outfile = os.path.join(out_path, '20200723_GRCh38_p13_regions.bed')
with open(outfile, 'w') as dump:
    _ = dump.write('#')
    all_regions.to_csv(dump, sep='\t', index=False, header=True)



Modeled_regions_for_GRCh38.tsv
GRCh38_p13_chromXY_PAR.tsv
ucsc_cytoband.bed
